베이지안 최적화로 파라미터 탐색 후 그나마 가장 괜찮게 판단된 모델(만약 거의 비슷하거나 차이가 유의하지 않을 정도로 판단된다면 베이스 모델로 하기 → 이상치는 잘못 기입한 값이 아닌 이상 자연적인 현상으로 처리하지 않는 것이 가장 현상을 잘 반영할 것이기 때문).

# 0. 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display as dp
import warnings
import matplotlib
import matplotlib.pyplot as plt

# 스케일링 => KNN은 거리 기반이므로 이상치에 민감하니 StandardScaler 사용
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix,f1_score, accuracy_score
from time import time
from skopt import BayesSearchCV

warnings.filterwarnings('ignore')
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [2]:
# 검증과 테스트 데이터셋은 피처 스케일링되어 사용되므로 먼저 정의하였다.
validation =pd.read_csv('../data/validation_encoding.csv', index_col=0)
test =pd.read_csv('../data/test_encoding.csv', index_col=0)

X_val, y_val = validation.iloc[:,:-1], validation.iloc[:,-1] # 그대로 사용됨. 
X_test, y_test = test.iloc[:,:-1], test.iloc[:,-1] # 그대로 사용됨..

dp(validation,test)
dp(X_val, y_val.to_frame() , X_test, y_test.to_frame())

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3,TARGET
73378,3,4.0,10,750,22.0,3.711225,0.000000,0.0,0.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,2
71524,8,0.0,10,329,12.0,2.734892,0.000000,0.0,0.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,1
38781,3,0.0,4,649,10.0,1.294901,0.000000,0.0,0.0,335.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
35212,10,5.0,6,628,20.0,1.337039,0.000000,0.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
56852,8,2.0,6,1846,36.0,2.158072,415.666667,0.0,0.0,205.0,...,1.0,0.0,0.0,0.0,0.0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23050,9,6.0,6,2148,38.0,2.184994,3.125000,0.0,0.0,220.0,...,1.0,0.0,0.0,0.0,0.0,1,0,0,0,0
36543,4,6.0,1,1827,51.0,2.204480,0.000000,0.0,0.0,215.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0
70132,11,4.0,2,1349,36.0,1.873360,1.142857,0.0,0.0,285.0,...,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0
58097,9,0.0,12,329,12.0,3.002181,19.625000,0.0,0.0,165.0,...,1.0,0.0,0.0,0.0,0.0,1,1,0,1,0


,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3,TARGET
12635,9,3.0,10,1050,32.0,1.254874,0.000000,0.0,0.000000,105.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
22195,5,5.0,10,1447,41.0,1.494139,5.142857,0.0,0.000000,220.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
42119,11,6.0,4,1348,6.0,2.063934,0.000000,0.0,0.000000,30.0,...,0.0,0.0,0.0,1.0,0.0,1,0,0,0,2
34954,1,2.0,1,746,21.0,1.746713,22.000000,35.0,48.333333,210.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
65122,4,3.0,2,426,13.0,2.079781,0.555556,0.0,0.000000,220.0,...,1.0,0.0,0.0,0.0,0.0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41734,11,5.0,9,936,28.0,0.650350,0.000000,0.0,0.000000,320.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
16135,6,1.0,8,1938,52.0,2.386494,24.250000,0.0,0.000000,200.0,...,1.0,0.0,0.0,0.0,0.0,0,0,1,0,1
65530,1,2.0,9,1148,29.0,0.446058,28.875000,34.0,144.000000,345.0,...,0.0,0.0,0.0,1.0,0.0,0,0,1,1,1
80594,1,5.0,6,36,46.0,2.517878,23.000000,15.8,22.666667,310.0,...,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0


,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3
73378,3,4.0,10,750,22.0,3.711225,0.000000,0.0,0.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
71524,8,0.0,10,329,12.0,2.734892,0.000000,0.0,0.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
38781,3,0.0,4,649,10.0,1.294901,0.000000,0.0,0.0,335.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
35212,10,5.0,6,628,20.0,1.337039,0.000000,0.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
56852,8,2.0,6,1846,36.0,2.158072,415.666667,0.0,0.0,205.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23050,9,6.0,6,2148,38.0,2.184994,3.125000,0.0,0.0,220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0,0
36543,4,6.0,1,1827,51.0,2.204480,0.000000,0.0,0.0,215.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
70132,11,4.0,2,1349,36.0,1.873360,1.142857,0.0,0.0,285.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
58097,9,0.0,12,329,12.0,3.002181,19.625000,0.0,0.0,165.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1,1,0,1


,TARGET
73378,2
71524,1
38781,0
35212,0
56852,1
...,...
23050,0
36543,0
70132,0
58097,0


,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3
12635,9,3.0,10,1050,32.0,1.254874,0.000000,0.0,0.000000,105.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
22195,5,5.0,10,1447,41.0,1.494139,5.142857,0.0,0.000000,220.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
42119,11,6.0,4,1348,6.0,2.063934,0.000000,0.0,0.000000,30.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1,0,0,0
34954,1,2.0,1,746,21.0,1.746713,22.000000,35.0,48.333333,210.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
65122,4,3.0,2,426,13.0,2.079781,0.555556,0.0,0.000000,220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41734,11,5.0,9,936,28.0,0.650350,0.000000,0.0,0.000000,320.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
16135,6,1.0,8,1938,52.0,2.386494,24.250000,0.0,0.000000,200.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0,1,0
65530,1,2.0,9,1148,29.0,0.446058,28.875000,34.0,144.000000,345.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0,0,1,1
80594,1,5.0,6,36,46.0,2.517878,23.000000,15.8,22.666667,310.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0


,TARGET
12635,1
22195,1
42119,2
34954,0
65122,0
...,...
41734,1
16135,1
65530,1
80594,0


In [3]:
# 피처 스케일링에 사용될 scaler
scaler = MinMaxScaler()

# base model의 학습 데이터
base_train = pd.read_csv('../data/base_train_encoding.csv', index_col=0)
X_train_base, y_train_base = base_train.iloc[:,:-1], base_train.iloc[:,-1]

X_train_base_scaled = scaler.fit_transform(X_train_base)

# 어떤 모델이건 사용될 검증/테스트 데이터셋 # 이게 아니라는 것. 훈련데이터셋이 바뀔 때마다 새롭게 transform 해야함. 이건 그냥 base_model의 경우임.
X_val_base_scaled = scaler.transform(X_val)
X_test_base_scaled = scaler.transform(X_test)

X_train_base_scaled.shape # pd.DataFrame(X_train_base_scaled, index = X_train_base.index, columns = X_train_base.columns)

(50643, 21)

___

# 1. 베이스 모델
- 이상치 처리는 하지 않고 피처 스케일링만 한 데이터셋으로 학습 후 확인 

In [4]:
from time import time
start = time()

knn = KNeighborsClassifier()

# 적절한 k값을 위한 과정
# from sklearn.model_selection import GridSearchCV, randomizedSearchCV # GridSearchCV는 모든 경우를 테스트하기 때문에 속도가 느림. 그러나 bayesizedSearchCV는 일부만 조합해서 최적의 값을 못 찾을지도

# scikit-optimize의 베이지안 최적화를 통해 적은 시도로 빠르게 구해볼 예정
from skopt import BayesSearchCV
params = {
    'n_neighbors' : (1,30),
    'weights' : ['uniform'],
    'p' : [2]
} # 탐색할 k값 범위 설정
# grid_search = GridSearchCV(knn, param_grid, cv = 5) # 5겹 교차 검증 너무 오래걸려서 사용 X
bayes_search = BayesSearchCV(knn, params, cv = 5, n_iter = 50, verbose = 2) # n_iter : 파라미터 조합 수
bayes_search.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')
k_knn_base = bayes_search.best_params_
print(k_knn_base)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ................n_neighbors=9, p=2, weights=uniform; total time=   7.4s
[CV] END ................n_neighbors=9, p=2, weights=uniform; total time=   7.7s
[CV] END ................n_neighbors=9, p=2, weights=uniform; total time=   7.5s
[CV] END ................n_neighbors=9, p=2, weights=uniform; total time=   7.6s
[CV] END ................n_neighbors=9, p=2, weights=uniform; total time=   7.8s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=27, p=2, weights=uniform; total time=   7.4s
[CV] END ...............n_neighbors=27, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=27, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=27, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=27, p=2, weights=uniform; total time=   7.8s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ....

[CV] END ................n_neighbors=1, p=2, weights=uniform; total time=   5.1s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=18, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=18, p=2, weights=uniform; total time=   7.8s
[CV] END ...............n_neighbors=18, p=2, weights=uniform; total time=   7.8s
[CV] END ...............n_neighbors=18, p=2, weights=uniform; total time=   7.8s
[CV] END ...............n_neighbors=18, p=2, weights=uniform; total time=   7.9s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   7.8s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   7.9s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   7.8s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   7.8s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time

[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   8.0s
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   8.2s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   7.9s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   8.1s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   8.2s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   8.3s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   8.7s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ................n_neighbors=6, p=2, weights=uniform; total time=   8.6s
[CV] END ................n_neighbors=6, p=2, weights=uniform; total time=   7.9s
[CV] END ................n_neighbors=6, p=2, weights=uniform; total time=   8.2s
[CV] END ................n_neighbors=6, p=2, weights=uniform; total time

In [5]:
# knn 모델 학습
start = time()

knn_base = KNeighborsClassifier(n_neighbors=k_knn_base['n_neighbors'],
                               weights = k_knn_base['weights'],
                               p = k_knn_base['p'])
knn_base.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = knn_base.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 0.00399470329284668
              precision    recall  f1-score   support

           0       0.54      0.74      0.62      7276
           1       0.54      0.37      0.44      5052
           2       0.43      0.33      0.37      4553

    accuracy                           0.52     16881
   macro avg       0.50      0.48      0.48     16881
weighted avg       0.51      0.52      0.50     16881

[[5369  893 1014]
 [2268 1858  926]
 [2350  719 1484]]
0.5160239322315029
0.5160239322315029


___

# 2. IQR_삭제 데이터셋
- 울타리 바깥 범위의 데이터를 삭제한 데이터셋으로 학습 후 확인

In [6]:
IQR_delete = pd.read_csv('../data/IQR_delete_encoding.csv',index_col=0)
X_train_IQR_delete, y_train_IQR_delete = IQR_delete.iloc[:,:-1], IQR_delete.iloc[:,-1]

# scaler는 새롭게 사용
scaler = MinMaxScaler()
X_train_IQR_delete_scaled = scaler.fit_transform(X_train_IQR_delete)
X_val_IQR_delete_scaled = scaler.transform(X_val)
X_test_IQR_delete_scaled = scaler.transform(X_test)

In [7]:
start = time()

knn = KNeighborsClassifier()

# 적절한 k값을 위한 과정
params = {
    'n_neighbors' : (1,30),
    'weights' : ['uniform'],
    'p' : [2]
} # 탐색할 k값 범위 설정
bayes_search =  BayesSearchCV(knn, params, cv = 5, n_iter = 50, verbose = 2) # n_iter : 파라미터 조합 수
bayes_search.fit(X_train_IQR_delete_scaled, y_train_IQR_delete)
print(f'IQR_삭제 모델 파라미터 찾기 : {time()-start}')
k_IQR_delete = bayes_search.best_params_
print(k_IQR_delete)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=25, p=2, weights=uniform; total time=   3.8s
[CV] END ...............n_neighbors=25, p=2, weights=uniform; total time=   3.9s
[CV] END ...............n_neighbors=25, p=2, weights=uniform; total time=   3.9s
[CV] END ...............n_neighbors=25, p=2, weights=uniform; total time=   3.9s
[CV] END ...............n_neighbors=25, p=2, weights=uniform; total time=   3.8s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=19, p=2, weights=uniform; total time=   3.9s
[CV] END ...............n_neighbors=19, p=2, weights=uniform; total time=   3.8s
[CV] END ...............n_neighbors=19, p=2, weights=uniform; total time=   3.8s
[CV] END ...............n_neighbors=19, p=2, weights=uniform; total time=   3.9s
[CV] END ...............n_neighbors=19, p=2, weights=uniform; total time=   3.8s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ....

[CV] END ...............n_neighbors=15, p=2, weights=uniform; total time=   4.1s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   3.8s
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   3.9s
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   4.0s
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   3.9s
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   3.8s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=28, p=2, weights=uniform; total time=   4.0s
[CV] END ...............n_neighbors=28, p=2, weights=uniform; total time=   3.9s
[CV] END ...............n_neighbors=28, p=2, weights=uniform; total time=   4.0s
[CV] END ...............n_neighbors=28, p=2, weights=uniform; total time=   4.2s
[CV] END ...............n_neighbors=28, p=2, weights=uniform; total time

[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   3.8s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   3.7s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   3.7s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   3.9s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   3.8s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   3.8s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   3.7s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=18, p=2, weights=uniform; total time=   3.7s
[CV] END ...............n_neighbors=18, p=2, weights=uniform; total time=   3.9s
[CV] END ...............n_neighbors=18, p=2, weights=uniform; total time=   3.8s
[CV] END ...............n_neighbors=18, p=2, weights=uniform; total time

In [8]:
# knn 모델 학습
start = time()
knn_IQR_delete = KNeighborsClassifier(n_neighbors=k_IQR_delete['n_neighbors'],
                               weights = k_IQR_delete['weights'],
                               p = k_IQR_delete['p'])
knn_IQR_delete.fit(X_train_IQR_delete_scaled, y_train_IQR_delete)
print(f'IQR_삭제 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = knn_IQR_delete.predict(X_val_IQR_delete_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

IQR_삭제 모델 : 0.0040035247802734375
              precision    recall  f1-score   support

           0       0.54      0.74      0.62      7276
           1       0.53      0.35      0.42      5052
           2       0.42      0.32      0.37      4553

    accuracy                           0.51     16881
   macro avg       0.50      0.47      0.47     16881
weighted avg       0.50      0.51      0.49     16881

[[5404  849 1023]
 [2310 1766  976]
 [2377  699 1477]]
0.5122326876369884
0.5122326876369884


___

# 3. IQR_가중치 데이터셋
- 울타리 바깥 범위의 데이터에 대해 거리의 역수를 가중치로 적용. 해당 데이터셋으로 학습 후 확인

In [9]:
IQR_weight = pd.read_csv('../data/IQR_weight_encoding.csv', index_col=0)

scaler = MinMaxScaler()
X_train_IQR_weight, y_train_IQR_weight = IQR_weight.iloc[:,:-1], IQR_weight.iloc[:,-1]
X_train_IQR_weight_scaled = scaler.fit_transform(X_train_IQR_weight)
X_val_IQR_weight_scaled = scaler.transform(X_val)
X_test_IQR_weight_scaled = scaler.transform(X_test)

In [10]:
start = time()

knn = KNeighborsClassifier()

# 적절한 k값을 위한 과정
params = {
    'n_neighbors' : (1,30),
    'weights' : ['uniform'],
    'p' : [2]
} # 탐색할 k값 범위 설정
bayes_search = BayesSearchCV(knn, params, cv = 5, n_iter = 50, verbose = 2)
bayes_search.fit(X_train_IQR_weight_scaled, y_train_IQR_weight)
print(f'IQR_가중치 모델 파라미터 찾기 : {time()-start}')
k_knn_IQR_weight = bayes_search.best_params_
print(k_knn_IQR_weight)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=21, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=21, p=2, weights=uniform; total time=   7.7s
[CV] END ...............n_neighbors=21, p=2, weights=uniform; total time=   7.7s
[CV] END ...............n_neighbors=21, p=2, weights=uniform; total time=   7.8s
[CV] END ...............n_neighbors=21, p=2, weights=uniform; total time=   7.9s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   7.6s
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   7.6s
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   7.7s
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   7.8s
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   7.8s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ....

[CV] END ...............n_neighbors=24, p=2, weights=uniform; total time=   7.8s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=17, p=2, weights=uniform; total time=   7.5s
[CV] END ...............n_neighbors=17, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=17, p=2, weights=uniform; total time=   7.7s
[CV] END ...............n_neighbors=17, p=2, weights=uniform; total time=   7.7s
[CV] END ...............n_neighbors=17, p=2, weights=uniform; total time=   7.7s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ................n_neighbors=3, p=2, weights=uniform; total time=   5.5s
[CV] END ................n_neighbors=3, p=2, weights=uniform; total time=   5.6s
[CV] END ................n_neighbors=3, p=2, weights=uniform; total time=   5.5s
[CV] END ................n_neighbors=3, p=2, weights=uniform; total time=   7.4s
[CV] END ................n_neighbors=3, p=2, weights=uniform; total time

[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   7.7s
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   7.9s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   7.7s
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   7.7s
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   7.8s
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   7.9s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=15, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=15, p=2, weights=uniform; total time=   7.7s
[CV] END ...............n_neighbors=15, p=2, weights=uniform; total time=   7.7s
[CV] END ...............n_neighbors=15, p=2, weights=uniform; total time

In [11]:
# knn 모델 학습
start = time()

knn_IQR_weight = KNeighborsClassifier(n_neighbors=k_knn_IQR_weight['n_neighbors'],
                               weights = k_knn_IQR_weight['weights'],
                               p = k_knn_IQR_weight['p'])

knn_IQR_weight.fit(X_train_IQR_weight_scaled, y_train_IQR_weight)
print(f'IQR_가중치 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = knn_IQR_weight.predict(X_val_IQR_weight_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

IQR_가중치 모델 : 0.005006313323974609
              precision    recall  f1-score   support

           0       0.53      0.75      0.62      7276
           1       0.54      0.33      0.41      5052
           2       0.43      0.33      0.37      4553

    accuracy                           0.51     16881
   macro avg       0.50      0.47      0.47     16881
weighted avg       0.51      0.51      0.49     16881

[[5440  820 1016]
 [2378 1688  986]
 [2418  630 1505]]
0.5114033528819383
0.5114033528819383


___

# 4. Isolation Forest_삭제 데이터셋
- Isolation Forest로 이상치 탐지 후 이상치로 판단된 관측치는 삭제한 데이터셋으로 학습 후 확인

In [12]:
Isolation_Forest_delete = pd.read_csv('../data/Isolation_Forest_delete_encoding.csv', index_col=0)

scaler = MinMaxScaler()
X_train_Isolation_Forest_delete, y_train_Isolation_Forest_delete = Isolation_Forest_delete.iloc[:,:-1], Isolation_Forest_delete.iloc[:,-1]
X_train_Isolation_Forest_delete_scaled = scaler.fit_transform(X_train_Isolation_Forest_delete)
X_val_Isolation_Forest_delete_scaled = scaler.transform(X_val)
X_test_Isolation_Forest_delete_scaled = scaler.transform(X_test)

In [13]:
start = time()


# 적절한 k값을 위한 과정
params = {
    'n_neighbors' : (1,30),
    'weights' : ['uniform'],
    'p' : [2]
} # 탐색할 k값 범위 설정
bayes_search = BayesSearchCV(knn, params, cv = 5, n_iter = 50, verbose = 2)
bayes_search.fit(X_train_Isolation_Forest_delete_scaled, y_train_Isolation_Forest_delete)
print(f'Isolation Forest_삭제 모델 파라미터 찾기 : {time()-start}')
k_knn_Isolation_Forest_delete = bayes_search.best_params_
print(k_knn_Isolation_Forest_delete)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ................n_neighbors=6, p=2, weights=uniform; total time=   5.6s
[CV] END ................n_neighbors=6, p=2, weights=uniform; total time=   5.6s
[CV] END ................n_neighbors=6, p=2, weights=uniform; total time=   5.6s
[CV] END ................n_neighbors=6, p=2, weights=uniform; total time=   5.6s
[CV] END ................n_neighbors=6, p=2, weights=uniform; total time=   5.6s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   5.6s
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   5.6s
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   5.7s
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   5.6s
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   5.6s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ....

[CV] END ...............n_neighbors=25, p=2, weights=uniform; total time=   5.7s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ................n_neighbors=1, p=2, weights=uniform; total time=   3.8s
[CV] END ................n_neighbors=1, p=2, weights=uniform; total time=   3.8s
[CV] END ................n_neighbors=1, p=2, weights=uniform; total time=   3.8s
[CV] END ................n_neighbors=1, p=2, weights=uniform; total time=   3.7s
[CV] END ................n_neighbors=1, p=2, weights=uniform; total time=   3.8s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   5.7s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   5.6s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   5.6s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time=   5.7s
[CV] END ...............n_neighbors=29, p=2, weights=uniform; total time

[CV] END ................n_neighbors=2, p=2, weights=uniform; total time=   4.0s
[CV] END ................n_neighbors=2, p=2, weights=uniform; total time=   4.1s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=13, p=2, weights=uniform; total time=   6.1s
[CV] END ...............n_neighbors=13, p=2, weights=uniform; total time=   5.6s
[CV] END ...............n_neighbors=13, p=2, weights=uniform; total time=   5.6s
[CV] END ...............n_neighbors=13, p=2, weights=uniform; total time=   5.7s
[CV] END ...............n_neighbors=13, p=2, weights=uniform; total time=   5.6s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=10, p=2, weights=uniform; total time=   5.6s
[CV] END ...............n_neighbors=10, p=2, weights=uniform; total time=   5.5s
[CV] END ...............n_neighbors=10, p=2, weights=uniform; total time=   5.7s
[CV] END ...............n_neighbors=10, p=2, weights=uniform; total time

In [14]:
# knn 모델 학습
start = time()

knn_Isolation_Forest_delete = KNeighborsClassifier(n_neighbors=k_knn_Isolation_Forest_delete['n_neighbors'],
                               weights = k_knn_Isolation_Forest_delete['weights'],
                               p = k_knn_Isolation_Forest_delete['p'])
knn_Isolation_Forest_delete.fit(X_train_Isolation_Forest_delete_scaled, y_train_Isolation_Forest_delete)
print(f'Isolation Forest_삭제 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = knn_Isolation_Forest_delete.predict(X_val_Isolation_Forest_delete_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

Isolation Forest_삭제 모델 : 0.003989696502685547
              precision    recall  f1-score   support

           0       0.53      0.74      0.62      7276
           1       0.53      0.35      0.42      5052
           2       0.43      0.33      0.37      4553

    accuracy                           0.51     16881
   macro avg       0.50      0.47      0.47     16881
weighted avg       0.51      0.51      0.49     16881

[[5408  861 1007]
 [2343 1756  953]
 [2386  679 1488]]
0.5125288786209348
0.5125288786209348


___

# 5. Isolation Forest_가중치 데이터셋
- Isolation Forest로 이상치 탐지 후 이상치로 판단된 관측치에는 MinMaxScale을 진행한 이상치 점수를 가중치로 적용. 해당 데이터셋으로 학습 후 확인

In [15]:
Isolation_Forest_weight = pd.read_csv('../data/Isolation_Forest_weight_encoding.csv', index_col=0)

scaler = MinMaxScaler()
X_train_Isolation_Forest_weight, y_train_Isolation_Forest_weight = Isolation_Forest_weight.iloc[:,:-1], Isolation_Forest_weight.iloc[:,-1]
X_train_Isolation_Forest_weight_scaled = scaler.fit_transform(X_train_Isolation_Forest_weight)
X_val_Isolation_Forest_weight_scaled = scaler.transform(X_val)
X_test_Isolation_Forest_weight_scaled = scaler.transform(X_test)

In [16]:
from time import time
from skopt import BayesSearchCV

In [17]:
start = time()

knn = KNeighborsClassifier()

# 적절한 k값을 위한 과정
params = {
    'n_neighbors' : (1,30),
    'weights' : ['uniform'],
    'p' : [2]
} # 탐색할 k값 범위 설정
bayes_search = BayesSearchCV(knn, params, cv = 5, n_iter = 50, verbose = 2)
bayes_search.fit(X_train_Isolation_Forest_weight_scaled, y_train_Isolation_Forest_weight)
print(f'Isolation Forest_가중치 모델 파라미터 찾기 : {time()-start}')
k_knn_Isolation_Forest_weight = bayes_search.best_params_
print(k_knn_Isolation_Forest_weight)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=19, p=2, weights=uniform; total time=   7.4s
[CV] END ...............n_neighbors=19, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=19, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=19, p=2, weights=uniform; total time=   9.3s
[CV] END ...............n_neighbors=19, p=2, weights=uniform; total time=   7.9s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   7.4s
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   7.6s
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   7.6s
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   7.7s
[CV] END ................n_neighbors=8, p=2, weights=uniform; total time=   7.7s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ....

[CV] END ...............n_neighbors=27, p=2, weights=uniform; total time=   7.8s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=10, p=2, weights=uniform; total time=   7.5s
[CV] END ...............n_neighbors=10, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=10, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=10, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=10, p=2, weights=uniform; total time=   7.8s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   7.5s
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   7.7s
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   7.7s
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=30, p=2, weights=uniform; total time

[CV] END ...............n_neighbors=13, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=13, p=2, weights=uniform; total time=   7.7s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=11, p=2, weights=uniform; total time=   7.3s
[CV] END ...............n_neighbors=11, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=11, p=2, weights=uniform; total time=   7.5s
[CV] END ...............n_neighbors=11, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=11, p=2, weights=uniform; total time=   7.8s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...............n_neighbors=16, p=2, weights=uniform; total time=   7.3s
[CV] END ...............n_neighbors=16, p=2, weights=uniform; total time=   7.6s
[CV] END ...............n_neighbors=16, p=2, weights=uniform; total time=   7.5s
[CV] END ...............n_neighbors=16, p=2, weights=uniform; total time

In [18]:
# knn 모델 학습
start = time()

knn_Isolation_Forest_weight = KNeighborsClassifier(n_neighbors=k_knn_Isolation_Forest_weight['n_neighbors'],
                               weights = k_knn_Isolation_Forest_weight['weights'],
                               p = k_knn_Isolation_Forest_weight['p'])

knn_Isolation_Forest_weight.fit(X_train_Isolation_Forest_weight_scaled, y_train_Isolation_Forest_weight)
print(f'Isolation Forest_가중치 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = knn_Isolation_Forest_weight.predict(X_val_Isolation_Forest_weight_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

Isolation Forest_가중치 모델 : 0.003995180130004883
              precision    recall  f1-score   support

           0       0.54      0.74      0.62      7276
           1       0.53      0.36      0.43      5052
           2       0.43      0.32      0.37      4553

    accuracy                           0.51     16881
   macro avg       0.50      0.48      0.47     16881
weighted avg       0.51      0.51      0.50     16881

[[5398  873 1005]
 [2275 1822  955]
 [2371  713 1469]]
0.5147206919021385
0.5147206919021385


___

모든 경우에서 성능 차이가 크지 않았음. 따라서 파라미터 강화는 베이스 모델로 진행할 것임.